# K-Means Clustering

## Chapter 6

K-Means clustering is a form of unsupervised learning where the goal is to discover clusters - groups - not known ahead of time within the data. We do not have the target, but instead will form clusters and then analyze them for any meaning we can derive.

In [1]:
from __future__ import annotations
from typing import TypeVar, Generic, List, Sequence
from copy import deepcopy
from functools import partial
from random import uniform
from statistics import mean, pstdev
from dataclasses import dataclass

## Function to Calculate Z-Scores

In [2]:
def zscores(original: Sequence[float]) -> List[float]:
    """
    Calculate zscores of a sequence.
    """
    avg: float = mean(original)
    std: float = pstdev(original)
        
    # Cannot divide by 0
    if std == 0:
        return original
    
    # Apply the zscore formalize to normalize
    return [(x - avg) / std for x in original]

## Define a Dataclass

In [3]:
from math import sqrt

class DataPoint:
    def __init__(self, initial: Iterable[float]) -> None:
        self._originals: Tuple[float, ...] = tuple(initial)
        self.dimensions: Tuple[float, ...] = tuple(initial)
            
    @property
    def num_dimensions(self) -> int:
        return len(self.dimensions)
    
    def distance(self, other: DataPoint) -> float:
        """
        Distance between one datapoint and another. Defined as the Euclidean distance.
        """
        combined: Iterable[Tuple[float, float]] = zip(self.dimensions, other.dimensions)
        differences: List[float] = [(x - y) ** 2 for x, y in combined]
        return sqrt(sum(differences))
    
    def __eq__(self, other: object) -> bool:
        if not isinstance(other, DataPoint):
            return NotImplemented
        return self.dimensions == other.dimensions
    
    def __repr__(self) -> str:
        return self._originals.__repr__()

In [4]:
dp = DataPoint(initial=(9.8, 5.6, 11.2))
dp2 = DataPoint(initial=(11.3, 4.5, 5.4))

print(dp)

(9.8, 5.6, 11.2)


In [5]:
print(dp==dp2)

False


In [6]:
dp.distance(dp2)

6.090976933136423

In [7]:
dp2.distance(dp)

6.090976933136423

In [8]:
zscores(dp._originals)
zscores(dp2._originals)

[0.39223227027636814, -1.3728129459672884, 0.9805806756909196]

[1.4036791557351331, -0.8510495668630337, -0.5526295888720998]

## Normalization

K-Means is a distance based algorithm, which means it requires the scale of data to be the same for each data point. One way to do this is through calculating the zscores of each datapoint (also called the standard score) which takes the average and divides by the standard deviation to put points on the same scale.

# K-Means Algorithm

1. Initialize "k" empty clusters
2. Normalize all data points
3. Create random centroids for each cluster
4. Assign ach data point to the cluster of the centroid to which it is closest
5. Recalculate each centroid based on the data points assigned to it
6. Continue until no data points are re-assigned or until the maximum number of iterations is reached.

In [106]:
from random import sample

Point = TypeVar("Point", bound=DataPoint)


class KMeans(Generic[Point]):
    @dataclass
    class Cluster:
        points: List[Point]
        centroid: DataPoint

    def __init__(self, k: int, points: List[Point]) -> None:
        if k < 1:  # Must have at least one cluster!
            raise ValueError("k must be >= 1")
        self._points: List[Point] = points
        self._zscore_normalize()

        # Initial clusters with random centroids
        self._clusters: List[KMeans.Cluster] = [KMeans.Cluster([], centroid) for centroid in self._random_centroids(k=k)]

    @property
    def _centroids(self) -> List[DataPoint]:
        return [x.centroid for x in self._clusters]

    def _dimension_slice(self, dimension: int) -> List[float]:
        """
        Return a column of data.
        """
        return [x.dimensions[dimension] for x in self._points]

    def _zscore_normalize(self) -> None:
        """
        Normalize each column to be zscores.
        """
        # List of lists to hold zscored columns
        zscored: List[List[float]] = [[] for _ in range(len(self._points))]

        # Iterate through each column index
        for dimension in range(self._points[0].num_dimensions):
            # Extrac the column
            dimension_slice: List[float] = self._dimension_slice(dimension)
            for index, zscore in enumerate(zscores(dimension_slice)):
                # Calculate the zscores of the column and keep track
                zscored[index].append(zscore)

        # Replace the points with their zscored equivalents
        for i in range(len(self._points)):
            self._points[i].dimensions = tuple(zscored[i])

    def _random_centroids(self, k: int) -> List[DataPoint]:
        """
        Select random points from data to serve as inital centroids.
        """
        return sample(self._points, k=k)
        

    def _assign_clusters(self) -> None:
        """
        Assign points to closest cluster measured by distance to centroid.
        """
        for point in self._points:
            # Find the closest centroid to the point
            closest: DataPoint = min(
                self._centroids, key=partial(DataPoint.distance, point)
            )
            idx: int = self._centroids.index(closest)
            cluster: KMeans.Cluster = self._clusters[idx]
            cluster.points.append(point)

    def _generate_centroids(self) -> None:
        """
        Calculate the centroids of each cluster using the mean of the points assigned to the cluster.
        """
        for cluster in self._clusters:
            if len(cluster.points) == 0:  # Do not move centroid if no points assigned
                continue
            means: List[float] = []
            # Iterate through each dimension index
            for dimension in range(cluster.points[0].num_dimensions):
                # Extract the dimension (column)
                # Cannot just use dimension slice because only concerned with points in cluster
                dimension_slice: List[float] = [
                    p.dimensions[dimension] for p in cluster.points
                ]
                # Calculate the mean of the dimension
                means.append(mean(dimension_slice))
            # Update the centroid of the cluster
            cluster.centroid = DataPoint(means)

    def run(self, max_iterations: int = 100) -> List[KMeans.Cluster]:
        """
        Run the KMeans clustering algorithm to find clusters.
        """
        for iteration in range(max_iterations):
            # Clear out the clusters
            for cluster in self._clusters:
                cluster.points.clear()
            # Assign points to closest cluster centroid
            self._assign_clusters()
            old_centroids: List[DataPoint] = deepcopy(self._centroids)
            # Calculate new centroids based on assign points to clusters
            self._generate_centroids()
            # If no points have been reassigned
            if old_centroids == self._centroids:
                print(f"Converged after {iteration} iterations.")
                return self._clusters
        return self._clusters

In [107]:
from random import randrange

point_list = [DataPoint([randrange(10), randrange(10), randrange(10)]) for _ in range(10)]
kmeans = KMeans(2, point_list)

In [108]:
kmeans._points
kmeans._clusters

[(7, 3, 3),
 (7, 2, 6),
 (9, 9, 3),
 (4, 4, 1),
 (5, 0, 3),
 (8, 5, 9),
 (4, 0, 1),
 (7, 6, 7),
 (2, 5, 0),
 (0, 7, 1)]

[KMeans.Cluster(points=[], centroid=(9, 9, 3)),
 KMeans.Cluster(points=[], centroid=(0, 7, 1))]

In [109]:
kmeans._zscore_normalize()

for point in kmeans._points:
    print(point.dimensions)

(0.6331130843152798, -0.39667014528604044, -0.14106912317171966)
(0.6331130843152798, -0.7572793682733499, 0.9169493006161779)
(1.3779520070391387, 1.7669851926378162, -0.14106912317171966)
(-0.48414529977050813, -0.03606092229873097, -0.8464147390303179)
(-0.1117258384085788, -1.4784978142479688, -0.14106912317171966)
(1.0055325456772093, 0.3245483006885785, 1.974967724404075)
(-0.48414529977050813, -1.4784978142479688, -0.8464147390303179)
(0.6331130843152798, 0.685157523675888, 1.269622108545477)
(-1.2289842224943668, 0.3245483006885785, -1.1990875469596172)
(-1.9738231452182253, 1.0457667466631975, -0.8464147390303179)


In [110]:
mean(kmeans._dimension_slice(0))
mean(kmeans._dimension_slice(1))
mean(kmeans._dimension_slice(2))

1.6653345369377347e-17

-2.151057110211241e-17

5.551115123125783e-18

In [111]:
pstdev(kmeans._dimension_slice(0))
pstdev(kmeans._dimension_slice(1))
pstdev(kmeans._dimension_slice(2))

1.0

1.0

1.0

## Test KMeans with Simple Points

In [113]:
point_list = [DataPoint([randrange(10), randrange(10), randrange(10)]) for _ in range(10)]
kmeans = KMeans(2, point_list)
result = kmeans.run()

Converged after 2 iterations.


In [114]:
for index, cluster in enumerate(result):
    print(f"Cluster {index} Points: {cluster.points} Centroid: {cluster.centroid}.")

Cluster 0 Points: [(2, 0, 9), (4, 3, 2), (3, 0, 1), (2, 2, 0)] Centroid: (-1.0277777777777775, -1.0572506251888203, -0.23320381162846943).
Cluster 1 Points: [(7, 8, 8), (4, 8, 1), (7, 4, 3), (6, 9, 3), (5, 7, 7), (6, 5, 3)] Centroid: (0.6851851851851853, 0.7048337501258803, 0.15546920775231285).


## KMeans on Governors

Try to find clusters of governors looking at age and longitude.

In [116]:
class Governor(DataPoint):
    """
    Class for representing a governor of a state.
    """
    def __init__(self, longitude: float, age: float, state: str) -> None:
        super().__init__([longitude, age])
        self.longitude = longitude
        self.age = age
        self.state = state
        
    def __repr__(self) -> str:
        return f"{self.state}: (longitude: {self.longitude}, age: {self.age})"

In [117]:
governors: List[Governor] = [
    Governor(-86.79113, 72, "Alabama"),
    Governor(-152.404419, 66, "Alaska"),
    Governor(-111.431221, 53, "Arizona"),
    Governor(-92.373123, 66, "Arkansas"),
    Governor(-119.681564, 79, "California"),
    Governor(-105.311104, 65, "Colorado"),
    Governor(-72.755371, 61, "Connecticut"),
    Governor(-75.507141, 61, "Delaware"),
    Governor(-81.686783, 64, "Florida"),
    Governor(-83.643074, 74, "Georgia"),
    Governor(-157.498337, 60, "Hawaii"),
    Governor(-114.478828, 75, "Idaho"),
    Governor(-88.986137, 60, "Illinois"),
    Governor(-86.258278, 49, "Indiana"),
    Governor(-93.210526, 57, "Iowa"),
    Governor(-96.726486, 60, "Kansas"),
    Governor(-84.670067, 50, "Kentucky"),
    Governor(-91.867805, 50, "Louisiana"),
    Governor(-69.381927, 68, "Maine"),
    Governor(-76.802101, 61, "Maryland"),
    Governor(-71.530106, 60, "Massachusetts"),
    Governor(-84.536095, 58, "Michigan"),
    Governor(-93.900192, 70, "Minnesota"),
    Governor(-89.678696, 62, "Mississippi"),
    Governor(-92.288368, 43, "Missouri"),
    Governor(-110.454353, 51, "Montana"),
    Governor(-98.268082, 52, "Nebraska"),
    Governor(-117.055374, 53, "Nevada"),
    Governor(-71.563896, 42, "New Hampshire"),
    Governor(-74.521011, 54, "New Jersey"),
    Governor(-106.248482, 57, "New Mexico"),
    Governor(-74.948051, 59, "New York"),
    Governor(-79.806419, 60, "North Carolina"),
    Governor(-99.784012, 60, "North Dakota"),
    Governor(-82.764915, 65, "Ohio"),
    Governor(-96.928917, 62, "Oklahoma"),
    Governor(-122.070938, 56, "Oregon"),
    Governor(-77.209755, 68, "Pennsylvania"),
    Governor(-71.51178, 46, "Rhode Island"),
    Governor(-80.945007, 70, "South Carolina"),
    Governor(-99.438828, 64, "South Dakota"),
    Governor(-86.692345, 58, "Tennessee"),
    Governor(-97.563461, 59, "Texas"),
    Governor(-111.862434, 70, "Utah"),
    Governor(-72.710686, 58, "Vermont"),
    Governor(-78.169968, 60, "Virginia"),
    Governor(-121.490494, 66, "Washington"),
    Governor(-80.954453, 66, "West Virginia"),
    Governor(-89.616508, 49, "Wisconsin"),
    Governor(-107.30249, 55, "Wyoming"),
]

In [119]:
kmeans: KMeans[Governor] = KMeans(2, governors)
result = kmeans.run()

for index, cluster in enumerate(result):
    print(f"Cluster {index}: {cluster.points}\n")

Converged after 4 iterations.
Cluster 0: [Arizona: (longitude: -111.431221, age: 53), Connecticut: (longitude: -72.755371, age: 61), Delaware: (longitude: -75.507141, age: 61), Illinois: (longitude: -88.986137, age: 60), Indiana: (longitude: -86.258278, age: 49), Iowa: (longitude: -93.210526, age: 57), Kansas: (longitude: -96.726486, age: 60), Kentucky: (longitude: -84.670067, age: 50), Louisiana: (longitude: -91.867805, age: 50), Maryland: (longitude: -76.802101, age: 61), Massachusetts: (longitude: -71.530106, age: 60), Michigan: (longitude: -84.536095, age: 58), Mississippi: (longitude: -89.678696, age: 62), Missouri: (longitude: -92.288368, age: 43), Montana: (longitude: -110.454353, age: 51), Nebraska: (longitude: -98.268082, age: 52), Nevada: (longitude: -117.055374, age: 53), New Hampshire: (longitude: -71.563896, age: 42), New Jersey: (longitude: -74.521011, age: 54), New Mexico: (longitude: -106.248482, age: 57), New York: (longitude: -74.948051, age: 59), North Carolina: (lon

## KMeans depends on initial cluster assignments. Run multiple times because results are different each time!

### KMeans++ Used to Initialize centroids based on probability distance to every point instead of pure randomness

Another option is to choose centroids ahead of time based on domain data or user input.

Variants on KMeans use other measures for the location of a centroid besides the means such as K-Medians or K-Mediods which uses an actual data point as the center of each cluster.

# Usage

KMeans can be used when we have unlabeled data and we want to find meaningful clusters within the data. For example, we might have a number of buildings, and want to find characteristics in common across the buildings. We could also try and group volunteers by their characteristics to find clusters of volunteers.